# Fussing with Full Resolution

In [11]:
# imports
import os
import glob
import xarray

import numpy as np
from scipy import stats
from scipy.interpolate import interp1d 

from gsw import conversions, density
import gsw

from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator 

import seaborn as sns

import pandas

from ocpy.utils import plotting

from cugn import grid_utils
from cugn import utils as cugn_utils
from cugn import io as cugn_io
from cugn import annualcycle

# Load CUGN

In [31]:
line = '90.0'
#
items = cugn_io.load_up(line)#, gextrem='low')
grid_extrem = items[0]
ds = items[1]

Loading: doxy_grid_line90.parquet
Loading: doxy_grid_line90.parquet
Found 29 unique clusters


# Examine a Mission and a few of its Profiles

In [43]:
mname = '20503001'
in_grid = (grid_extrem.mission.values == mname)
np.unique(grid_extrem.mission_profile.values[in_grid])
profile = 972

In [45]:
idx = (grid_extrem.mission.values == mname) & (grid_extrem.mission_profile.values == profile)
np.sum(idx)
grid_extrem.MLD.values[idx]

array([6.08593589])

In [39]:
grid_extrem

,depth,profile,row,col,doxy,mission,mission_profile,doxy_p,time,lon,...,MLD,cuti,beuti,N_p,chla_p,year,doy,dist,offset,cluster
776,0,32958,24,19,277.438868,16C06401,786,99.602386,2017-03-14 02:17:03,-118.227115,...,12.241844,0.139,0.370,80.914513,36.978131,2017,73,52.253342,-0.061142,0
777,0,32959,24,20,276.052619,16C06401,787,98.923077,2017-03-14 03:41:20,-118.217615,...,13.728747,0.139,0.370,71.076923,39.230769,2017,73,51.204435,-0.083858,0
782,0,32964,24,20,298.647665,16C06401,792,99.846154,2017-03-14 15:53:33,-118.129770,...,11.651772,0.081,0.411,80.307692,44.153846,2017,73,41.597092,-0.135953,0
783,0,32965,22,20,292.036344,16C06401,793,99.783550,2017-03-14 18:54:13,-118.106223,...,7.023053,0.081,0.411,99.783550,67.748918,2017,73,39.311737,0.349343,0
784,0,32966,22,19,309.040496,16C06401,794,99.756098,2017-03-14 21:51:29,-118.080510,...,7.242711,0.081,0.411,99.268293,52.926829,2017,73,37.025676,1.239828,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60503,3,38762,28,22,276.418914,18606401,425,99.626667,2018-08-09 14:45:33,-118.261740,...,7.547126,0.024,0.001,88.640000,27.413333,2018,221,55.366575,-1.200414,27
60505,3,38764,28,22,277.398417,18606401,427,99.733333,2018-08-09 17:29:48,-118.227913,...,7.607407,0.024,0.001,89.706667,28.533333,2018,221,51.843854,-0.915903,27
60506,3,38765,28,23,276.223865,18606401,428,99.491741,2018-08-09 19:10:34,-118.205372,...,7.987058,0.024,0.001,88.881830,31.130877,2018,221,49.579385,-0.583768,27
78917,4,38065,28,19,271.442412,18605501,601,98.073022,2018-08-27 23:46:22,-122.370410,...,28.762089,1.246,1.559,85.395538,58.012170,2018,239,505.510655,2.598540,-1


## Load up

In [18]:
high_path = os.path.join(os.getenv('OS_SPRAY'), 'CUGN', 'HighRes')
gfiles = glob.glob(os.path.join(high_path, f'SPRAY-FRSQ-{mname}-*.nc'))
hfile = gfiles[0]                   
#hfile
ds_high = xarray.open_dataset(hfile)
ds_high

<xarray.Dataset> Size: 203MB
Dimensions:                     (trajectory: 1, profile: 1004, obs: 1104091)
Coordinates:
  * trajectory                  (trajectory) |S19 19B b'sp030-20200513T1702'
  * profile                     (profile) float64 8kB 1.0 2.0 ... 1.004e+03
    time                        (profile) datetime64[ns] 8kB ...
    latitude                    (profile) float64 8kB ...
    latitude_uv                 (profile) float64 8kB ...
    longitude                   (profile) float64 8kB ...
    longitude_uv                (profile) float64 8kB ...
    time_uv                     (profile) datetime64[ns] 8kB ...
    depth                       (obs) float64 9MB ...
Dimensions without coordinates: obs
Data variables: (12/54)
    profile_obs_index           (obs) float64 9MB ...
    row_size                    (profile) int32 4kB ...
    time_qc                     (profile) float64 8kB ...
    latitude_qc                 (profile) float64 8kB ...
    latitude_uv_qc              (profile) float64 8kB ...
    longitude_qc                (profile) float64 8kB ...
    ...                          ...
    gps_end_qc_tests            (profile) float64 8kB ...
    wmo_id                      <U7 28B ...
    instrument_doxy             float64 8B ...
    instrument_ctd              float64 8B ...
    instrument_fchl             float64 8B ...
    platform_meta               float64 8B ...
Attributes: (12/65)
    Conventions:                   CF-1.8, COARDS, ACDD-1.3
    acknowledgement:               Funded by National Oceanic and Atmospheric...
    cdm_profile_variables:         profile,time,latitude,longitude
    cdm_trajectory_variables:      trajectory
    comment:                       This file contains data from the following...
    contributor_name:              Daniel Rudnick, Jennifer Sevadjian
    ...                            ...
    time_coverage_duration:        P0000-03-19T23:09:53
    time_coverage_end:             2020-09-02T16:12:26Z
    time_coverage_resolution:      P0000-00-00T00:00:04
    time_coverage_start:           2020-05-13T17:02:33Z
    title:                         Science Quality Full-Resolution Data from ...
    history:                       2020-09-14T16:18:20Z: KDZ readflasho63(gps...

In [34]:
my_obs = ds_high.profile_obs_index.values == profile
np.sum(my_obs)

np.int64(823)